<span style="font-size:24px; font-family:'Roboto'; font-weight:bold;">
Script to create new OBS IPF file.
</span><br>
It was discovered that some OBS are not assigned to the correct layer. That will be corrected via this script.

# 0. Options

In [1]:
import WS_Mdl as WS
import pandas as pd
import os
from os import listdir as LD, makedirs as MDs
from os.path import join as PJ, basename as PBN, dirname as PDN, exists as PE
import numpy as np
from scipy.spatial import cKDTree

In [2]:
import imod

In [3]:
os.getcwd()

'c:\\OD\\WS_Mdl\\code\\PrP\\make_OBS_IPF'

In [4]:
MdlN = 'NBr8'

In [5]:
MdlN_B = 'NBr7'

In [6]:
d_Pa = WS.get_MdlN_paths(MdlN)

In [7]:
def match_layers(row):
    ix = row['X']
    iy = row['Y']
    Flt_top = row['filtertoplevel']
    Flt_bot = row['filterbottomlevel']
    
    _, flat_idx = tree.query([ix, iy])
    i = flat_idx  # already aligned with .ravel()
    
    case1 = case2 = case3 = Thk1 = Thk2 = Thk3 = None
    case4, Thk4 = [], []

    for L in range(n_layers):
        top_L = TOP_vals[L, i]
        bot_L = BOT_vals[L, i]
        if top_L==bot_L: continue  # skip if no layer
        
        case1_bool = Flt_top >= top_L >= Flt_bot             # L_top within filter
        case2_bool = Flt_top >= bot_L >= Flt_bot             # L_bot within filter
        case3_bool = Flt_top <= top_L and Flt_bot >= bot_L   # filter within L
        case4_bool = Flt_top >= top_L and Flt_bot <= bot_L   # L fully within filter (multiple layers can be within the filter, hence the list)

        if case4_bool: case4.append(L), Thk4.append(top_L-bot_L)
        elif case1_bool: case1, Thk1 = L, (top_L - Flt_bot)
        elif case2_bool: case2, Thk2 = L, (Flt_top - bot_L)
        elif case3_bool: case3, Thk3 = L, (Flt_top - Flt_bot)

    matched = [k for k in (case1, case2, case3) if k is not None] + case4
    matched_1b = [k + 1 for k in matched]
    
    intersected = [k for k in (Thk1, Thk2, Thk3) if k is not None] + Thk4

    if matched:
        sh = min(matched)
        dp = max(matched)
        top_val = float(TOP_vals[sh, i])
        bot_val = float(BOT_vals[dp, i])
    else:
        top_val = bot_val = None

    return pd.Series({
        'case1_L': case1 + 1 if case1 is not None else None,
        'case2_L': case2 + 1 if case2 is not None else None,
        'case3_L': case3 + 1 if case3 is not None else None,
        'case4_L': [k + 1 for k in case4],
        'L_match': matched_1b,
        'Itsc_Thk': intersected,
        'TOP_L_match': top_val,
        'BOT_L_match': bot_val,
        'match_distance': float(_)  # from tree.query
    })

# 1. Read previous IPF file.
B is NBr5, but NBr7 is the same, but with corrected column names (helps with PP automation etc.).

In [8]:
DF_IPF_B = WS.read_IPF_Spa(f'../../../models/NBr/In/OBS/{MdlN_B}/ijkset_selectie_{MdlN_B}.ipf')

# 2. Read TOP and BOT arrays 

In [9]:
Pa_TOP = r'C:\OD\WS_Mdl\models\NBr\In\TOP'
l_TOP = [PJ(Pa_TOP, i) for i in LD(Pa_TOP) if '.idf'== i[-4:]]
TOP = imod.formats.idf.open(l_TOP, pattern="{name}_L{layer}_")

In [10]:
Pa_BOT = r'C:\OD\WS_Mdl\models\NBr\In\BOT'
l_BOT = [PJ(Pa_BOT, i) for i in LD(Pa_BOT) if '.idf'== i[-4:]]
BOT = imod.formats.idf.open(l_BOT, pattern="{name}_L{layer}_")

# 3. Match L based on TOP and BOT values (compared to filter top and bot)

In [11]:
xv, yv = np.meshgrid(TOP.coords['x'].values, TOP.coords['y'].values)
flat_coords = np.column_stack((xv.ravel(), yv.ravel()))
tree = cKDTree(flat_coords)

In [12]:
# Preload TOP/BOT values
TOP_vals = TOP.values.reshape((TOP.shape[0], -1))
BOT_vals = BOT.values.reshape((BOT.shape[0], -1))
n_layers = TOP.shape[0]

In [13]:
DF_IPF_B = DF_IPF_B.join(DF_IPF_B.apply(match_layers, axis=1))

In [14]:
DF_IPF_B[DF_IPF_B['L_match'].apply(lambda x: len(x) > 0)]

,X,Y,L,Id,code,filterno,surfacelevel,filtertoplevel,filterbottomlevel,path,case1_L,case2_L,case3_L,case4_L,L_match,Itsc_Thk,TOP_L_match,BOT_L_match,match_distance
0,130879.0,404056.0,5,2001_1,2001,1,-0.69,7.82,6.82,../NBr5/ijkset_selectie/2001_1,5.0,3.0,NaN,[4],"[5, 3, 4]","[0.6799998, 0.30999994, 0.010000229]",8.740000,3.260000,29.614186
1,130879.0,404056.0,7,2001_2,2001,2,5.13,-10.09,-11.09,../NBr5/ijkset_selectie/2001_2,NaN,NaN,7.0,[],[7],[1.0],3.250000,-24.370001,29.614186
2,111628.0,405726.0,1,2013_1,2013,1,1.92,-0.53,-1.53,../NBr5/ijkset_selectie/2013_1,NaN,NaN,1.0,[],[1],[1.0],1.530000,-2.092080,32.557641
3,126388.0,411199.0,5,2014_1,2014,1,-0.32,-1.30,-2.30,../NBr5/ijkset_selectie/2014_1,7.0,5.0,NaN,[6],"[7, 5, 6]","[0.13999987, 0.85000014, 0.00999999]",-0.310000,-23.799999,62.008064
4,126391.0,411132.0,5,2015_1,2015,1,-0.03,-1.11,-2.11,../NBr5/ijkset_selectie/2015_1,NaN,NaN,5.0,[],[5],[0.9999999999999998],-0.310000,-2.150000,44.777226
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7243,181076.0,375269.0,5,WOOL003_1,WOOL003,1,25.58,22.70,21.70,../NBr5/ijkset_selectie/WOOL003_1,NaN,NaN,5.0,[],[5],[1.0],22.780001,18.959999,32.202484
7244,181150.0,375100.0,5,WOOL004_1,WOOL004,1,28.34,24.64,23.64,../NBr5/ijkset_selectie/WOOL004_1,5.0,3.0,NaN,[4],"[5, 3, 4]","[0.8100014, 0.1800003, 0.0099983215]",27.469999,20.209999,50.000000
7245,182883.0,374584.0,3,WOOL006_1,WOOL006,1,27.76,26.16,25.16,../NBr5/ijkset_selectie/WOOL006_1,NaN,NaN,3.0,[],[3],[1.0],27.049999,24.870001,47.381431
7246,181081.0,375259.0,3,WOOL007_1,WOOL007,1,26.28,24.84,23.84,../NBr5/ijkset_selectie/WOOL007_1,NaN,NaN,3.0,[],[3],[1.0],25.309999,22.790001,32.280025


# 4. Review and edit DF_IPF

In [15]:
print(f"{DF_IPF_B[DF_IPF_B['L_match'].apply(lambda x: len(x) > 0)].shape[0]} out of {DF_IPF_B.shape[0]} features were matched to layers. I cross checked this in QGIS - the number of points within the large model output raster was 6255, which is really close. I'll assign the old layer to the points with no match. ")

6234 out of 7248 features were matched to layers. I cross checked this in QGIS - the number of points within the large model output raster was 6255, which is really close. I'll assign the old layer to the points with no match. 


In [16]:
DF_IPF = DF_IPF_B.copy()
# DF_test['L_match'] = DF_test['L_match'].astype(str).str.split(', ')

In [17]:
DF_IPF = DF_IPF.explode(['L_match', 'Itsc_Thk']).reset_index(drop=True)

In [ ]:
DF_IPF.loc[DF_IPF['L_match'].isna()]

,X,Y,L,Id,code,filterno,surfacelevel,filtertoplevel,filterbottomlevel,path,case1_L,case2_L,case3_L,case4_L,L_match,Itsc_Thk,TOP_L_match,BOT_L_match,match_distance
430,92995.00,410405.00,1,9011_1,9011,1,4.87,0.55,-0.45,../NBr5/ijkset_selectie/9011_1,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN,63.639610
431,93816.00,410407.00,1,9014_1,9014,1,4.69,0.79,-0.21,../NBr5/ijkset_selectie/9014_1,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN,54.817880
432,95243.00,410443.00,1,9016_1,9016,1,-0.13,0.64,-0.36,../NBr5/ijkset_selectie/9016_1,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN,9.899495
452,88441.35,405058.52,1,9051_1,9051,1,2.69,0.94,-0.06,../NBr5/ijkset_selectie/9051_1,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN,12.141371
550,81889.20,338438.10,7,1-0262_1,1-0262,1,-9999.00,1.30,-2.70,../NBr5/ijkset_selectie/1-0262_1,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN,40.966450
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9299,158836.00,328611.80,37,N_64bol_3_2,N/64bol/3,2,-9999.00,32.90,30.90,../NBr5/ijkset_selectie/N_64bol_3_2,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN,40.684641
9300,158836.00,328611.80,37,N_64bol_3_3,N/64bol/3,3,-9999.00,32.90,30.90,../NBr5/ijkset_selectie/N_64bol_3_3,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN,40.684641
9301,95148.80,331092.10,7,N_73_1_1,N/73/1,1,-9999.00,4.92,2.92,../NBr5/ijkset_selectie/N_73_1_1,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN,42.117099
9302,161515.40,324640.80,7,N_78_1_1,N/78/1,1,-9999.00,40.75,38.75,../NBr5/ijkset_selectie/N_78_1_1,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN,35.802235


In [19]:
DF_IPF['L'] = DF_IPF.apply(lambda x: x['L_match'] if pd.notna(x.get('L_match')) else x['L'], axis=1)

In [21]:
DF_IPF = DF_IPF[[*DF_IPF.columns[:3], 'Itsc_Thk', *DF_IPF.columns.drop('Itsc_Thk')[3:]]]

In [22]:
DF_IPF

,X,Y,L,Itsc_Thk,Id,code,filterno,surfacelevel,filtertoplevel,filterbottomlevel,path,case1_L,case2_L,case3_L,case4_L,L_match,TOP_L_match,BOT_L_match,match_distance
0,130879.0,404056.0,5,0.68,2001_1,2001,1,-0.69,7.82,6.82,../NBr5/ijkset_selectie/2001_1,5.0,3.0,NaN,[4],5,8.740000,3.260000,29.614186
1,130879.0,404056.0,3,0.31,2001_1,2001,1,-0.69,7.82,6.82,../NBr5/ijkset_selectie/2001_1,5.0,3.0,NaN,[4],3,8.740000,3.260000,29.614186
2,130879.0,404056.0,4,0.01,2001_1,2001,1,-0.69,7.82,6.82,../NBr5/ijkset_selectie/2001_1,5.0,3.0,NaN,[4],4,8.740000,3.260000,29.614186
3,130879.0,404056.0,7,1.0,2001_2,2001,2,5.13,-10.09,-11.09,../NBr5/ijkset_selectie/2001_2,NaN,NaN,7.0,[],7,3.250000,-24.370001,29.614186
4,111628.0,405726.0,1,1.0,2013_1,2013,1,1.92,-0.53,-1.53,../NBr5/ijkset_selectie/2013_1,NaN,NaN,1.0,[],1,1.530000,-2.092080,32.557641
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9673,182883.0,374584.0,3,1.0,WOOL006_1,WOOL006,1,27.76,26.16,25.16,../NBr5/ijkset_selectie/WOOL006_1,NaN,NaN,3.0,[],3,27.049999,24.870001,47.381431
9674,181081.0,375259.0,3,1.0,WOOL007_1,WOOL007,1,26.28,24.84,23.84,../NBr5/ijkset_selectie/WOOL007_1,NaN,NaN,3.0,[],3,25.309999,22.790001,32.280025
9675,180742.0,375926.0,5,0.290001,WOOL008_1,WOOL008,1,26.09,24.65,23.65,../NBr5/ijkset_selectie/WOOL008_1,5.0,3.0,NaN,[4],5,26.330000,19.680000,25.298221
9676,180742.0,375926.0,3,0.699999,WOOL008_1,WOOL008,1,26.09,24.65,23.65,../NBr5/ijkset_selectie/WOOL008_1,5.0,3.0,NaN,[4],3,26.330000,19.680000,25.298221


In [23]:
DF_IPF = DF_IPF.iloc[:, :11]

In [24]:
DF_IPF['Id'] = DF_IPF['Id'] + '_L' + DF_IPF['L'].astype(str)

In [25]:
MDs(f'../../../models/NBr/In/OBS/{MdlN}', exist_ok=True)

In [27]:
imod.formats.ipf.write(f'../../../models/NBr/In/OBS/{MdlN}/ijkset_selectie_{MdlN}.ipf', DF_IPF, indexcolumn=len(DF_IPF.columns), assoc_ext='txt', nodata=1e+20)